## Raw notebook with debug functions and multithreading for speeding up on my local machine

In [ ]:
import pandas as pd
pd.options.display.max_columns = 500
import matplotlib.pyplot as plt
import glob
import cv2
import os
import time
from multiprocessing import Process

In [ ]:
df = pd.read_excel('phones.xlsx')
df = df.sort_values(by=['Release Date'])
df.columns = df.columns.str.replace(' ', '')
df.head()

In [ ]:
to_graph = ['ScreenSize', 'ResolutionWidth', 'ResolutionHeight', 'BackCamera', 'FrontCamera', 'Memory', 'Battery', 'weight', 's2b', 'storagemin', 'storagemax', 'corecount', 'clockspeed', 'GB4S', 'GB4M', 'GB5S', 'GB5M']
to_graph_smaller = [['ScreenSize', 'ResolutionWidth', 'ResolutionHeight', 'BackCamera'],['FrontCamera', 'Memory','Battery', 'weight'], ['s2b','storagemin','storagemax','corecount'],['clockspeed','GB4S', 'GB4M','GB5S','GB5M'] ]
threads = []
for i in to_graph_smaller:
    print(i)
    p = Process(target=graph_from_list, args = (df, i, 10,))
    threads.append(p)
for j in range(len(threads)):
    threads[j].start()
for j in range(len(threads)):
    threads[j].join()

In [ ]:
to_graph = ['ResolutionHeight', 'BackCamera', 'FrontCamera',
            'Memory', 'Battery', 'weight', 's2b', 'storagemin', 'storagemax', 'corecount',
            'clockspeed', 'GB4S', 'GB4M', 'GB5S', 'GB5M']
def get_units(category):
    if(category == 'ScreenSize'):
        return("Screen Size (diagonal) in inches")
    if(category == 'ResolutionWidth'):
        return("Screen Resolution (width) in pixels")
    if(category == 'ResolutionHeight'):
        return("Screen Resolution (height) in pixels")
    if(category == 'BackCamera'):
        return("Rear Camera Resolution (megapixels)")
    if(category == 'FrontCamera'):
        return("Front Camera Resolution (megapixels)")
    if(category == 'Memory'):
        return("Phone Memory (RAM in Megabytes)")
    if(category == 'Battery'):
        return("Battery Capacity (milliamp hours)")
    if(category == 'weight'):
        return("Phone Weight (grams)")
    if(category == 's2b'):
        return("Phone Screen to Body Ratio (percentage)")
    if(category == 'storagemin'):
        return("Phone Storage minimum (Gigabytes)")
    if(category == 'storagemax'):
        return("Phone Storage maximum (Gigabytes)")
    if(category == 'corecount'):
        return("Phone Core Count")
    if(category == 'clockspeed'):
        return("Phone maximum clockspeed (megahertz)")
    if(category == 'GB4S'):
        return("Geekbench 4 Single Core Score")
    if(category == 'GB4M'):
        return("Geekbench 4 Multi Core Score")
    if(category == 'GB5S'):
        return("Geekbench 5 Single Core Score")
    if(category == 'GB5M'):
        return("Geekbench 5 Multi Core Score")
    
# pass in df, what all you want to graph and how many to display on graph at one time (max), saves files in images directory
def graph_from_list(df, to_graph=to_graph, num_to_display=10):
    last_len = 0
    ts = time.time()
    print(ts)
    colors = []
    for g in to_graph:
        tts = time.time()
        for y in range(2007, 2020):
            for m in range(1,13):
                for d in range(1,31):
                    if(m == 2 and d > 28):
                        t = 0
                    else:
                        dt = str(y) + '-' + str(m) + '-' + str(d)
                        time_df = df[df['ReleaseDate'] < dt].sort_values(by=[g, 'ReleaseDate'])
                        data = time_df[g].to_list()
                        names = time_df['Name'].to_list()
                        mfgs = time_df['Brand'].to_list()
                        
                        
                        if(len(data) > last_len):
                            colors = []
                            for mfg in mfgs:
                                if(mfg == 'Apple'):
                                    colors.append("#ababab")
                                if(mfg == 'Samsung'):
                                    colors.append('#3e70fa')
                                if(mfg == 'Google'):
                                    colors.append('#3cba54')
                                elif(mfg != 'Apple' and mfg != 'Samsung' and mfg != 'Google'):
                                    colors.append('#996b32')
                        
                        if(len(data) > last_len):
                            labels = []
                            for i in range(len(data)):
                                labels.append(i * -1)
                            plt.barh(labels,data, tick_label=names, color=colors)
                            axes = plt.gca()
                            axes.set_xlim([0,(df[g].max())*1.1])
                            axes.set_ylim([len(data)*-1+num_to_display+0.5,len(data)*-1 + 0.5])
                            plt.title(dt)
                            plt.gcf().subplots_adjust(left=0.1)
                            plt.gcf().set_size_inches(16,9)
                            plt.xlabel(get_units(g))
                            plt.ylabel('Device')
                            path = 'images/g' + g + dt +'.jpg'
                            for l in range(len(data)):
                                if(labels[l] < len(data)*-1+num_to_display+0.5 and labels[l] > len(data)*-1 + 0.5):
                                    plt.text(data[l], labels[l], data[l])
                            plt.savefig(path, pad_inches=0.5)
                            plt.cla()
    #                         plt.show()
                            last_len = len(data)
        print(tts-time.time())
        break
        for y in range(2007, 2020):
            for m in range(1,13):
                ds = [1,14,28]
                for d in ds:
                    dt = str(y) + '-' + str(m) + '-' + str(d)
                    time_df = df[df['ReleaseDate'] < dt].sort_values(by=[g, 'ReleaseDate'])
                    data = time_df[g].to_list()
                    names = time_df['Name'].to_list()
                    mfgs = time_df['Brand'].to_list()

                    
                    if(len(data) > last_len):
                        colors = []
                        for mfg in mfgs:
                            if(mfg == 'Apple'):
                                colors.append("#ababab")
                            if(mfg == 'Samsung'):
                                colors.append('#3e70fa')
                            if(mfg == 'Google'):
                                colors.append('#3cba54')
                            elif(mfg != 'Apple' and mfg != 'Samsung' and mfg != 'Google'):
                                colors.append('#996b32')

#                         if(len(data) > last_len):
                    labels = []
                    for i in range(len(data)):
                        labels.append(i)
                    plt.barh(labels,data, tick_label=names, color=colors)
                    axes = plt.gca()
                    axes.set_xlim([0,(df[g].max())*1.1])
#                     if(len(data) < 10):
#                         axes.set_ylim([len(data)*-1+num_to_display+0.5,len(data)*-1 + 0.5])
                    plt.title(dt)
                    plt.gcf().subplots_adjust(left=0.1)
                    plt.gcf().set_size_inches(16,9)
                    plt.xlabel(get_units(g))
                    plt.ylabel('Device')
                    path = 'images/' + g + dt +'_recap.jpg'
                    for l in range(len(data)):
                            plt.text(data[l], labels[l], data[l])
                    plt.savefig(path, pad_inches=0.5)
                    plt.cla()
#                         plt.show()
                    last_len = len(data)
        print(tts-time.time())       
        images = glob.glob('images/'+g+'*.jpg')
        images.sort(key=os.path.getmtime)
    #     print(images)
        out = cv2.VideoWriter('images/test_' + g + '.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 60, (1280,720))
        for image in images:
            cap = cv2.imread(image)
            cap = cv2.resize(cap, (1280,720))
            out.write(cap)
        out.release()
        print(tts-time.time())
    print(ts-time.time())
graph_from_list(df, to_graph, 10)

In [ ]:
ts = time.time()
for g in to_graph:
    tts = time.time()
    images = glob.glob('images/g'+g+'*.jpg')
#     print(images)
    images.sort(key=os.path.getmtime)
#     print(images)
    out = cv2.VideoWriter('images/test_g' + g + '.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 60, (1280,720))
    for image in images:
        cap = cv2.imread(image)
        cap = cv2.resize(cap, (1280,720))
        out.write(cap)
    out.release()
    print(tts-time.time())
    break
print(ts-time.time())